In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./modules")

import os
import argparse
import numpy as np
import matplotlib.pyplot as plt

from Potential import Potential
from Channel import Channel
from Scattering import Scattering
from ScatteringExp import ScatteringExp
from Grid import Grid
import RseSolver
from scipy.special import factorial2, spherical_jn
from plots.rc_params import *

In [8]:
potentialArgs = {"label": "minnesota", "kwargs": {"potId": 213}}
# lvalues = [0]
# channels = [Channel(S=0, L=l, LL=l, J=l, channel=0) for l in lvalues]  # only spin 0 

l = 2
channel = Channel(S=0, L=l, LL=l, J=l, channel=0)
potential = Potential(channel, **potentialArgs)
trainingLecList, testingLecList = Potential.getSampleLecs(potentialArgs["label"])

In [25]:
potentialArgs = {"label": "minnesota", "kwargs": {"potId": 213}}
l = 0
channel = Channel(S=0, L=l, LL=l, J=l, channel=0)
potential = Potential(channel, **potentialArgs)
E_MeV = 50
scattExp = ScatteringExp(E_MeV=E_MeV, potential=potential)
rmatch = 10
grid = Grid(1e-24, rmatch, numIntervals=1, numPointsPerInterval=600,
            type="linear", test=False)  # needs equidistant grids as of now
trainingLecList, testingLecList = Potential.getSampleLecs(potentialArgs["label"])

from NumerovEmulator import AffineGROM
# Test config for no greedy
emulator_pod = AffineGROM(potential=potential, channel=channel, scattExp=scattExp, 
                               num_snapshots_init=7, greedy_max_iter=5, num_snapshots_max=100,
                               pod=False, greedy=False, grid=grid, free_lecs=("V0",))

testing
